# Дистилляция BERT

Дообученная модель BERT показывает очень хорошее качество при решении множества NLP-задач. Однако, её не всегда можно применить на практике из-за того, что модель очень большая и работает дастаточно медленно. В связи с этим было придумано несколько способов обойти это ограничение.

Один из способов - `knowledge distillation`.

Суть метода заключается в следующем. Мы берём две модели - нашу обученную на решение конкретной задачи BERT (модель-учитель) и модель с более простой архитектурой (модель-ученик). Модель-ученик будет обучаться поведению модели-учителя: логиты Берта мы будем подавать модели-ученику в процессе её обучения.

В качестве модели-учителя возьмём уже обученную ранее модель, классифицирующую названия строительных товаров.

## Библиотеки

In [0]:
pip install transformers catboost

In [77]:
import os
import random
import numpy as np
import pandas as pd
import torch

from transformers import AutoConfig, AutoModelForSequenceClassification
from transformers import AutoTokenizer
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler

from catboost import Pool, CatBoostRegressor

from sklearn.metrics import classification_report
from tqdm.notebook import tqdm

SEED = 22
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device.type)
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))

cuda
Tesla P100-PCIE-16GB


## Загрузка токенизатора, модели, конфигурации

In [0]:
# config
config = AutoConfig.from_pretrained('/content/drive/My Drive/colab_data/leroymerlin/model/BERT_model')
# tokenizer
tokenizer = AutoTokenizer.from_pretrained('/content/drive/My Drive/colab_data/leroymerlin/model/BERT_model', pad_to_max_length=True)
# model
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/My Drive/colab_data/leroymerlin/model/BERT_model', config=config)

## Подготовка данных

In [0]:
category_index = {'Водоснабжение': 8,
 'Декор': 12,
 'Инструменты': 4,
 'Краски': 11,
 'Кухни': 15,
 'Напольные покрытия': 5,
 'Окна и двери': 2,
 'Освещение': 13,
 'Плитка': 6,
 'Сад': 9,
 'Сантехника': 7,
 'Скобяные изделия': 10,
 'Столярные изделия': 1,
 'Стройматериалы': 0,
 'Хранение': 14,
 'Электротовары': 3}
category_index_inverted = dict(map(reversed, category_index.items()))

In [0]:
df = pd.read_csv('/content/drive/My Drive/colab_data/leroymerlin/to_classifier.csv')
sentences = df.name.values
labels = [category_index[i] for i in df.category_1.values]

In [0]:
tokens = [tokenizer.encode(
        sent, 
        add_special_tokens=True, 
        max_length=24, 
        pad_to_max_length='right') for sent in sentences]

In [0]:
tokens_tensor = torch.tensor(tokens)
#labels_tensor = torch.tensor(labels)

In [0]:
BATCH_SIZE = 400
#full_dataset = TensorDataset(tokens_tensor, labels_tensor)
sampler = SequentialSampler(tokens_tensor)
dataloader = DataLoader(tokens_tensor, sampler=sampler, batch_size=BATCH_SIZE)

## Получение логитов BERT

In [84]:
train_logits = []
with torch.no_grad():
    model.to(device)
    for batch in tqdm(dataloader):
        batch = batch.to(device)
        outputs = model(batch)
        logits = outputs[0].detach().cpu().numpy()
        train_logits.extend(logits)

In [0]:
#train_logits = np.vstack(train_logits)

## Обучение ученика

Теперь возьмём мультирегрессионную модель от CatBoost и передадим ей все полученные логиты.

In [0]:
data_pool = Pool(tokens, train_logits)

In [0]:
distilled_model = CatBoostRegressor(iterations=2000, 
                          depth=4, 
                          learning_rate=.1, 
                          loss_function='MultiRMSE',
                          verbose=200)

In [88]:
distilled_model.fit(data_pool)

0:	learn: 11.6947874	total: 275ms	remaining: 9m 9s
200:	learn: 9.0435970	total: 47s	remaining: 7m
400:	learn: 8.2920608	total: 1m 32s	remaining: 6m 10s
600:	learn: 7.7736947	total: 2m 18s	remaining: 5m 22s
800:	learn: 7.3674586	total: 3m 4s	remaining: 4m 36s
1000:	learn: 7.0166625	total: 3m 51s	remaining: 3m 51s
1200:	learn: 6.7202548	total: 4m 38s	remaining: 3m 5s
1400:	learn: 6.4602129	total: 5m 25s	remaining: 2m 19s
1600:	learn: 6.2248947	total: 6m 12s	remaining: 1m 32s
1800:	learn: 6.0164036	total: 7m	remaining: 46.4s
1999:	learn: 5.8322141	total: 7m 46s	remaining: 0us


## Сравнение качества моделей

In [0]:
category_index_inverted = dict(map(reversed, category_index.items()))

### Метрики Берта:

In [90]:
print(classification_report(labels, np.argmax(train_logits, axis=1), target_names=category_index_inverted.values()))

                    precision    recall  f1-score   support

     Водоснабжение       0.94      0.88      0.91     13377
             Декор       1.00      0.40      0.57      2716
       Инструменты       1.00      0.40      0.58       540
            Краски       0.97      0.81      0.88     20397
             Кухни       0.96      0.91      0.93     29920
Напольные покрытия       1.00      0.56      0.72      2555
      Окна и двери       1.00      0.61      0.76      2440
         Освещение       0.98      0.92      0.95     30560
            Плитка       0.97      0.96      0.97     23922
               Сад       0.95      0.98      0.96     49518
        Сантехника       0.97      0.74      0.84     24245
  Скобяные изделия       0.85      0.93      0.89     15280
 Столярные изделия       0.58      0.95      0.72     30329
    Стройматериалы       0.98      0.67      0.80      8532
          Хранение       0.97      0.77      0.86      6237
     Электротовары       0.96      0.87

### Метрики модели-ученика:

In [0]:
tokens_pool = Pool(tokens)

distilled_predicted_logits = distilled_model.predict(tokens_pool, prediction_type='RawFormulaVal') # Probability

In [94]:
print(classification_report(labels, np.argmax(distilled_predicted_logits, axis=1), target_names=category_index_inverted.values()))

                    precision    recall  f1-score   support

     Водоснабжение       0.90      0.53      0.67     13377
             Декор       0.99      0.30      0.46      2716
       Инструменты       0.00      0.00      0.00       540
            Краски       0.97      0.61      0.75     20397
             Кухни       0.85      0.77      0.81     29920
Напольные покрытия       1.00      0.28      0.44      2555
      Окна и двери       0.96      0.30      0.45      2440
         Освещение       0.92      0.82      0.87     30560
            Плитка       0.94      0.86      0.90     23922
               Сад       0.85      0.86      0.86     49518
        Сантехника       0.91      0.55      0.68     24245
  Скобяные изделия       0.61      0.78      0.69     15280
 Столярные изделия       0.40      0.92      0.56     30329
    Стройматериалы       0.80      0.64      0.71      8532
          Хранение       0.93      0.50      0.65      6237
     Электротовары       0.88      0.24

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Как видим, качество модели-ученика немного хуже качества Берта, но скорее всего модель-ученик сможет иметь то же качество, если мы произведём тонкую настройку гиперпараметров.